# 4.8. Wnioskowanie na podstawie nauczonego modelu

Modeli regresji nie tworzymy dla sztuki, a chcemy najczęściej być w stanie przewidywać daną wartość w przyszłości, dla zupełnie nowych danych - najczęściej w zewnętrznych systemach. W związku z tym, dość istotną rzeczą jest możliwość przeniesienia nauczonego modelu. Większość dojrzałych narzędzi, w tym scikit-learn, umożliwiają *serializację* do formatu, który może być odczytany na zewnątrz.

In [1]:
import pandas as pd

In [2]:
wine_quality_df = pd.read_parquet("./data/wine-quality.parquet")
wine_quality_df.sample(n=5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,color_red,color_white,bound sulfur dioxide
4181,6.6,0.25,0.32,5.60,0.039,15.0,68.0,0.99163,2.96,0.52,11.1,6,0,1,53.0
1040,8.2,0.68,0.30,2.10,0.047,17.0,138.0,0.99500,3.22,0.71,10.8,4,0,1,121.0
1362,6.4,0.25,0.33,1.40,0.040,42.0,115.0,0.99060,3.19,0.48,11.3,7,0,1,73.0
2996,6.5,0.35,0.36,0.80,0.034,32.0,111.0,0.98942,3.11,0.50,12.1,8,0,1,79.0
4056,6.5,0.43,0.18,13.15,0.032,25.0,131.0,0.99565,3.23,0.51,10.7,5,0,1,106.0


In [3]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [5]:
pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("regressor", LinearRegression()),
])
pipeline.fit(wine_quality_df.drop("quality", axis="columns"), 
             wine_quality_df["quality"])

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('regressor',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

W momencie gdy przetestujemy już nasz model na niepełnym zbiorze, nic nie stoi na przeskodzie, aby nauczyć go jeszcze raz, tylko tym razem bez pomijania zbioru testowego. Zasadą jest to, iż ze wzrostem ilości danych, moc predykcyjna modelu powinna wzrastać.

## Serializacja modelu

Python ma standardową bibliotekę *pickle*, która służy do serializacji obiektów. Nie jest to rozwiązanie najwydajniejsze ani najbezpieczniejsze. Ładowanie kodu zapisanego poprzez ten moduł powoduje konieczność odpalenia pewnego kodu przy deserializacji - jeśli nie znamy źródła modelu, to możemy potencjalnie wykonać niebezpieczny kod.

Alternatywnym podejściem jest wykorzystanie modułu *joblib*, który umożliwia dokonanie serializacji z sposób bezpieczniejszy i typowo oszczędzając przy tym pamięć. 

In [6]:
import joblib

In [8]:
joblib.dump(pipeline, "./model/wine-quality-lr.joblib")

['./model/wine-quality-lr.joblib']

In [10]:
loaded_pipeline = joblib.load("./model/wine-quality-lr.joblib")
loaded_pipeline.predict(wine_quality_df.drop("quality", 
                                             axis="columns"))

array([5.56949282, 5.23283093, 5.83355143, ..., 5.92097627, 5.57319118,
       6.14957505])

## Monitorowanie jakości

Ważnym aspektem produkcyjnego uruchamiania jakiegokolwiek modelu ML, jest monitorowanie jego zachowania w fazie wdrożenia. Uwzględnia to zarówno aspekty czasowe, takie jak średni czas predykcji, ale również poprawność przewidywań. Najczęściej jednak nie znamy prawidłowych odpowiedzi, ale jeśli jesteśmy w stanie logować podane mu przykłady, to w przyszłości będzie możliwe ewaluowanie jak i douczenie stworzonego systemu.

**Wszystkie modele Machine Learning są nauczone na przeszłych danych i bazują na regularnościach obecnych historycznie, a niekoniecznie w przyszłości!**